In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
 
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0 #start of string
EOS_token = 1
 
 
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {} #単語の頻度
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # 辞書に登録してある単語の種類
 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
 
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
 
# Lowercase, trim, and remove non-letter characters
 
#テキストの前処理
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s #sentence

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
 
    # lang1, lang2は言語の洗濯(lang1が英語、lang2がフランス)
    lines = open('drive/My Drive/dataset/eng2fre/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
 
    # l = 'eng ¥t french'
    # lに対してユニコードや正規化の処理
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
 
    # 言語の翻訳の順番を変えるだけ
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        #初期化
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
 
    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 10 #最大単語数(句読点を含む)
 
#この文章から始まるデータに限定する
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
 
#上記2つの条件を満たすデータのみを抽出 
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)
 
 
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [97]:
filterPair('i am God')

False

In [98]:
def prepareData(lang1, lang2, reverse=False):
    #テキストファイルを読んで行に分割し、行をペアに分割する
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse) 
    print("Read %s sentence pairs" % len(pairs))
    #データの選別
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    #単語の辞書を作る
    for pair in pairs:
        input_lang.addSentence(pair[0]) #英語の1文
        output_lang.addSentence(pair[1]) #フランス語の一文
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs
 
 
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(len(pairs)) #shape->[10599, 2]
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
10599
['nous sommes en train de danser .', 'we re dancing .']


In [99]:
print(pairs[0])
print(pairs[0][0])
print(pairs[0][1])

['j ai ans .', 'i m .']
j ai ans .
i m .


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input_size：辞書の単語種類数、 hidden_size：分散表現の次元数
        self.embedding = nn.Embedding(input_size, hidden_size)
        #層を増やす実験
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
 
    def forward(self, input, hidden1, hidden2):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden1 = self.gru1(output, hidden1)
        output, hidden2 = self.gru2(output, hidden2)
        #出力と隠れ状態
        return output, hidden1, hidden2
 
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru1 = nn.GRU(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
 
    #hidedenはencoderの隠れ状態
    def forward(self, input, hidden1, hidden2):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden1 = self.gru(output, hidden1)
        output, hidden2 = self.gru(output, hidden2)
        output = self.softmax(self.out(output[0]))
        return output, hidden2
 
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
 
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru1 = nn.GRU(self.hidden_size, self.hidden_size)
        self.gru2 = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
 
    def forward(self, input, hidden1, hidden2, encoder_outputs):
        #input:[1], hidden:[1,1,256], encoder_outputs:[10,256]
        embedded = self.embedding(input).view(1, 1, -1) #[1, 1, 256]
        embedded = self.dropout(embedded)
        
        #加法注意によりattention_weightを求める。
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden1[0]), 1)), dim=1)
        
        #weightとエンコーダのアウトプットを用いる
        #bmmは各バッチごとに内積を出せる
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
 
        output = torch.cat((embedded[0], attn_applied[0]), 1) #[1, 512]
        output = self.attn_combine(output).unsqueeze(0) #[1, 1, 256]
 
        output = F.relu(output) #[1, 1, 256]
        output, hidden1 = self.gru1(output, hidden1) #[1, 1, 256], [1, 1, 256]
        output, hidden2 = self.gru2(output, hidden2)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden2, attn_weights
 
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    #各文を単語indexのならびに
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1) #[[2], [25], [9]....]の形に


def tensorsFromPair(pair):
    #pairは各文(0がフランス)
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [104]:
training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(2)]
training_pairs

[(tensor([[   6],
          [2678],
          [  66],
          [3690],
          [   5],
          [   1]], device='cuda:0'), tensor([[   2],
          [  16],
          [ 784],
          [ 739],
          [ 246],
          [2297],
          [   4],
          [   1]], device='cuda:0')), (tensor([[   6],
          [  11],
          [  14],
          [ 100],
          [ 233],
          [2361],
          [ 194],
          [2360],
          [   5],
          [   1]], device='cuda:0'), tensor([[   2],
          [   3],
          [1370],
          [1371],
          [   4],
          [   1]], device='cuda:0'))]

In [105]:
training_pairs[0][0].shape

torch.Size([6, 1])

In [0]:
teacher_forcing_ratio = 0.5
 
 
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden1 = encoder.initHidden()
    encoder_hidden2 = encoder.initHidden()
 
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
 
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
 
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
 
    loss = 0
    #input_tensorは[その文の単語数, 1]のテンソル、つまり1文のデータ
    for ei in range(input_length):
        #1文字のindexと隠れ状態を入れていく
        #input_tensor[ei]:[1], encoder_hidden:[1,1,256]
        encoder_output, encoder_hidden1, encoder_hidden2 = encoder(
            input_tensor[ei], encoder_hidden1, encoder_hidden2)
        #encoder_output:[1,1,256], encoder_hidden:[1,1,256]
        encoder_outputs[ei] = encoder_output[0, 0]
 
    decoder_input = torch.tensor([[SOS_token]], device=device)
 
    decoder_hidden1 = encoder_hidden1
    decoder_hidden2 = encoder_hidden2
 
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            #decoder_input:[1], decoder_hidden:[1,1,256], encoder_outputs:[10,256]
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden1, decoder_hidden2, encoder_outputs)
            #decoder_output:[1,2803], decoder_hidden:[1,1,256], decoder_attention:[1,10]
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
 
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden1, decoder_hidden2, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
 
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
 
    loss.backward()
 
    encoder_optimizer.step()
    decoder_optimizer.step()
 
    return loss.item() / target_length

In [0]:
import time
import math
 
 
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
 
 
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
 
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
 
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
 
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
 
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
 
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
 
    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
 
 
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden1 = encoder.initHidden()
        encoder_hidden2 = encoder.initHidden()
 
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
 
        for ei in range(input_length):
            encoder_output, encoder_hidden1, encoder_hidden2 = encoder(input_tensor[ei],
                                                     encoder_hidden1, encoder_hidden２)
            encoder_outputs[ei] += encoder_output[0, 0]
 
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
 
        decoder_hidden1 = encoder_hidden1
        decoder_hidden2 = encoder_hidden2
 
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
 
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden1, decoder_hidden2, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<eos>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
 
            decoder_input = topi.squeeze().detach()
 
        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [112]:
output_lang.n_words

2803

In [113]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 49s (- 25m 27s) (5000 6%) 2.7963
3m 35s (- 23m 23s) (10000 13%) 2.3174
5m 24s (- 21m 38s) (15000 20%) 2.0608
7m 13s (- 19m 53s) (20000 26%) 1.8263
9m 3s (- 18m 7s) (25000 33%) 1.6327
10m 54s (- 16m 21s) (30000 40%) 1.4666
12m 45s (- 14m 35s) (35000 46%) 1.3519
14m 36s (- 12m 47s) (40000 53%) 1.1859
16m 27s (- 10m 58s) (45000 60%) 1.0783
18m 19s (- 9m 9s) (50000 66%) 0.9360
20m 13s (- 7m 21s) (55000 73%) 0.8744
22m 6s (- 5m 31s) (60000 80%) 0.8061
23m 59s (- 3m 41s) (65000 86%) 0.7137
25m 52s (- 1m 50s) (70000 93%) 0.6521
27m 44s (- 0m 0s) (75000 100%) 0.5839


In [114]:
evaluateRandomly(encoder1, attn_decoder1)

> vous etes tout a fait attirant .
= you re quite attractive .
< you re quite attractive . <eos>

> ce sont tous des menteurs .
= they re all liars .
< they re all liars . <eos>

> vous n etes pas millionnaire .
= you re not a millionaire .
< you re not a millionaire . <eos>

> je suis contente d etre partie de boston .
= i m glad i left boston .
< i boston . <eos>

> ce n est rien qu une enfant .
= she is nothing but a child .
< she is nothing but a child . <eos>

> j en ai marre de ce temps humide .
= i m fed up with this wet weather .
< i am fed up with this wet weather . <eos>

> je ne suis jamais libre le dimanche .
= i am never free on sundays .
< i am never on sundays . <eos>

> nous ne sommes pas apparentees .
= we re unrelated .
< we re not related . <eos>

> vous etes tres efficace .
= you re very efficient .
< you re very efficient . <eos>

> vous etes l instituteur .
= you re the teacher .
< you re the teacher . <eos>



In [115]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)
 
    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<eos>'], rotation=90)
    ax.set_yticklabels([''] + output_words)
 
    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
 
    plt.show()
 
 
def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)
 
 
evaluateAndShowAttention("elle a cinq ans de moins que moi .")
 
#evaluateAndShowAttention("elle est trop petit .")
 
#evaluateAndShowAttention("je ne crains pas de mourir .")
 
#evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she s five years younger than me . <eos>
